In [4]:
!pip install wordcloud
!pip install senticnet
!pip install afinn
!pip install twython
!pip install langdetect
# !pip install cPickle
import nltk
nltk.download('stopwords')
nltk.download('sentiwordnet')
nltk.download('senticnet')
nltk.download('demo_liu_hu_lexicon')
nltk.download('SentimentIntensityAnalyzer')
nltk.download('wordnet')

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 61kB 1.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for afinn ... done
  Stored in directory: /Users/saurabhsharma/Library/Caches/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 122kB 2.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for twython 

True

In [1]:
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank
from nltk import bigrams, trigrams

In [2]:
from senticnet.senticnet import SenticNet

Loading all sentiment based lexicons

In [5]:
import senticnet
from afinn import Afinn
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.sentiment.util import demo_liu_hu_lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
import sklearn
import pandas as pd
import numpy as np
import json
import re
# import cPickle

In [6]:
from langdetect import detect


In [7]:
# HI Lex
with open('lexicons/HI_POS_dict.json', 'r') as f:
    pos_hi_dict = json.load(f)
with open('lexicons/HI_NEG_dict.json', 'r') as f:
    neg_hi_dict = json.load(f)

# EmoSentiNet
with open('lexicons/ESN_anger_dict.json', 'r') as f:
    esn_anger_dict = json.load(f)
with open('lexicons/ESN_sadness_dict.json', 'r') as f:
    esn_sad_dict = json.load(f)
with open('lexicons/ESN_joy_dict.json', 'r') as f:
    esn_joy_dict = json.load(f)
# with open('ESN_surprise_dict.json', 'r') as f:
#     esn_surprise_dict = json.load(f)
with open('lexicons/ESN_disgust_dict.json', 'r') as f:
    esn_disgust_dict = json.load(f)
with open('lexicons/ESN_fear_dict.json', 'r') as f:
    esn_fear_dict = json.load(f)


In [8]:
def clean_dict(dict):
  temp = {}
  for k,v in dict.items():
    cleaned = k.split('#')[0]
    temp[cleaned] = v
  return temp

In [9]:
pos_hi_dict = clean_dict(pos_hi_dict)
neg_hi_dict = clean_dict(neg_hi_dict)

In [10]:
def esn(s):
	bg = bigrams(s)
	tg = trigrams(s)
	score_esn_sad = 0
	score_esn_joy = 0
	score_esn_anger = 0
	score_esn_disgust = 0
	score_esn_fear = 0
	for b,t in zip(bg, tg):
		string1 = b[0] + " " + b[1]
		string2 = t[0] + " " + t[1] + " " + t[2]
		if string1 in esn_anger_dict.keys():
			score_esn_anger += 1
		if string2 in esn_anger_dict.keys():
			score_esn_anger += 1
		
		if string1 in esn_disgust_dict.keys():
			score_esn_disgust += 1
		if string2 in esn_disgust_dict.keys():
			score_esn_disgust += 1
		
		if string1 in esn_sad_dict.keys():
			score_esn_sad += 1
		if string2 in esn_sad_dict.keys():
			score_esn_sad += 1

		if string1 in esn_joy_dict.keys():
			score_esn_joy += 1
		if string2 in esn_joy_dict.keys():
			score_esn_joy += 1

		if string1 in esn_fear_dict.keys():
			score_esn_fear += 1
		if string2 in esn_fear_dict.keys():
			score_esn_fear += 1
	return (score_esn_anger, score_esn_disgust, score_esn_sad, score_esn_joy, score_esn_fear)


In [11]:
def liu_hu_lexicon(sentence):
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.
    :param sentence: a sentence whose polarity has to be classified.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
        elif word in opinion_lexicon.negative():
            neg_words += 1


    if pos_words > neg_words:
        return 1 # positive
    elif pos_words < neg_words:
        return -1 # negative
    elif pos_words == neg_words:
        return 0 # neutral

    return np.array(y)


In [13]:
filename = "lyrics.csv"
df = pd.read_csv(filename)

In [14]:
df = df[df['lyrics']!='instrumental'].dropna()

In [15]:
genres = df.genre.unique()

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
lyrics = df['lyrics']
vocab = set()
for lyric in lyrics:
  words = lyric.split()
  for word in words:
    vocab.add(lemmatizer.lemmatize(word.lower()))

In [0]:
len(vocab)

837079

In [0]:
def get_contexts_with_word(df2,word):
#   df2 = df.sample(100)
#   df2 = df2.dropna()
#   df2['lyrics'] = df2['lyrics'].replace(r'\n', ' ', regex = True)
  r = []
  ind = 0
  indices = []
  
  for i,row in df2.iterrows():
#     print(row)      
      l = row['lyrics']
      if (any(c.isalpha() for c in l) and detect(l) == 'en'):
          r.append(l.split())
          indices.append(row['index'])
      ind += 1


  k = 15
  she_phrases = []
  she_ind = []
  for i,v in enumerate(r):
    for ngram in nltk.ngrams(v, 2*k+1, pad_left=True, pad_right=True, left_pad_symbol=" ", right_pad_symbol = " "):
      if ngram[k].lower() == word:
          s = " ".join(ngram)
#           print('s', s)
          she_phrases.append((indices[i],s))
#           she_ind.append(indices)
          
  return (she_phrases,indices)

In [0]:
woman_insults = pd.read_csv('woman_insults.csv')

In [0]:
w_insults = woman_insults['insults'].tolist()
for i in range(len(w_insults)):
  w_insults[i] = w_insults[i].lower()

In [18]:
def get_sentiment_word_context(df2,contextp,word):
  for loc,context in contextp:
    context = context
    score_sn = 0
    score_swn_pos = 0
    sn = SenticNet()
    score_swn_neg = 0
    count_swn = 0
    score_hi_pos = 0
    score_hi_neg = 0
#     print(context)
    for word in context.split():
      # Sentic Net
      try:
        sentics = sn.sentics(word)
        score_sn_sensitivity = abs(float(sentics['sensitivity']))
        score_sn_pleasantness = float(sentics['pleasantness'])
      except:
        score_sn_sensitivity = 0
        score_sn_pleasantness = 0
  #     Sentic Word Net
      word_swn_lemmas = wn.synsets(word)
      if not not word_swn_lemmas:
        ans = swn.senti_synset(word_swn_lemmas[0].name()) #using 0 index because
        score_swn_pos += ans.pos_score()
        score_swn_neg += ans.neg_score()
        count_swn += 1
      # HI Score
      if word in pos_hi_dict.keys():
        score_hi_pos += pos_hi_dict[word]
      if word in neg_hi_dict.keys():
        score_hi_neg += neg_hi_dict[word]

    (score_esn_anger, score_esn_disgust, score_esn_sad, score_esn_joy, score_esn_fear) = esn(context)
#     print(score_esn_anger, score_esn_disgust, score_esn_sad, score_esn_joy, score_esn_fear)
    df2.at[loc,'score_esn_anger'] += score_esn_anger
    df2.at[loc,'score_esn_sad'] += score_esn_sad
    df2.at[loc,'score_esn_joy'] += score_esn_joy
    df2.at[loc,'score_esn_fear'] += score_esn_fear
    df2.at[loc,'score_esn_disgust'] += score_esn_disgust
    df2.at[loc,'score_hi_neg'] += score_hi_neg
    df2.at[loc,'score_hi_pos'] += score_hi_pos
    df2.at[loc,'score_swn_pos'] += score_swn_pos
    df2.at[loc,'score_swn_neg'] += score_swn_neg
    df2.at[loc,'score_sn_sensitivity'] += score_sn_sensitivity
    df2.at[loc,'score_sn_pleasantness'] += score_sn_pleasantness
  return df2


In [19]:
def add_cols(df):
  sLen = len(df)
  df['score_esn_anger'] = 0
  df['score_esn_sad'] = 0
  df['score_esn_joy'] = 0
  df['score_esn_fear'] = 0
  df['score_esn_disgust'] = 0
  df['score_hi_neg'] = 0
  df['score_hi_pos'] = 0
  df['score_swn_pos'] = 0
  df['score_swn_neg'] = 0
  df['score_sn_sensitivity'] = 0
  df['score_sn_pleasantness'] = 0

In [0]:
she_phrases,index = get_contexts_with_word(df,"she")

In [0]:
d_female = df.copy()
add_cols(d_female)

In [0]:
d_female = get_sentiment_word_context(d_female,he_phrases,"she")

In [0]:
df_male = df.copy()

In [0]:
add_cols(df_male)

In [0]:
he_phrases,index = get_contexts_with_word(df,"he")

In [0]:
d_male = get_sentiment_word_context(df_male,he_phrases,"he")

In [0]:
d_female = pd.to_csv('she_sentiment.csv')
d_male = pd.to_csv('he_sentiment.csv')